### Need to import a few tools/libraries to get started with our task

In [1]:
import requests
import http
import json
import csv

import requests
from urllib3.exceptions import InsecureRequestWarning

# Suppress only the single warning from urllib3 needed.
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

In [2]:
BACKBOXIP = "192.168.xxx.xxx"
BACKBOXUSER = 'username'
BACKBOXUSERPASS = 'password'

### To begin we need to get all of the needed data to run our script together. 

In [3]:
#information needed to reach out to backbox API
BackBoxBaseURL = "https://" + BACKBOXIP #example 192.168.1.2
BackBoxUser = BACKBOXUSER #enter your user name for BackBox
BackBoxPassword = BACKBOXUSERPASS #enter your user Password for BackBox
externalApiURL = BackBoxBaseURL + '/rest/data/api/'
internalApiURL = BackBoxBaseURL + '/rest/data/'

#custom information about files and jobs we will use
devices_to_upgrade_file = './Devices_to_upgrade.csv' #Path to a CSV with just external IDs in the first column
jobToExecute = "UpgradeJob" #Name of the task job we will execute in BackBox
jobFileName = "upgradefile.tgz" #this is the upgrade file we want to point to
jobFileToUpload = "./" + jobFileName #Path to the upgrade file we want to user for our upgrade



### Here we will import out list of external IDs from a CSV. These could come from anywhere in reality, this could be from your Device Database API or just from a CSV as in this example. Key here is that we get some sort of datastructure that will help us identify devices we want to operate on.

In [4]:
key = ''
devicesToUpgrade = []
with open(devices_to_upgrade_file, mode='r', newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile, delimiter = ' ', quotechar='|')
    first = True
    for item in reader:
        if first:
            key = item
            first = False
        else:
            devicesToUpgrade += item
print(devicesToUpgrade)

['FD1345', '2112']


In [5]:
session = requests.Session()

### Let's go get a connection to our backbox server. One way to authenticate here is to get our cookies and use them to maintain our session

In [6]:
#reach out to BackBox and get the login page
response = session.get(BackBoxBaseURL, verify=False)

### Now that we have our session cookies we can send username and password to login

In [7]:
headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
}

data = {
    'j_username': BackBoxUser,
    'j_password': BackBoxPassword,
}

response = session.post(BackBoxBaseURL+'/j_security_check', cookies=response.cookies, headers=headers, data=data, verify=False)

### First thing we will do now that we are logged in is upload our upgrade file

In [8]:
headers = {}
files = {"file": open(jobFileToUpload)}

filesUploaded = session.post(internalApiURL + "taskfile/0", cookies=response.cookies, headers=headers, verify=False, files=files)

In [9]:
fileUploaded = False
try:
    if filesUploaded.json()["id"] != "":
        fileUploaded = True

except:
    print("Your File Failed to Upload")
    fileUploaded = False
    exit()

### Now we will get data we need to add our devices to our intended job

In [10]:
headers = {}
devicesToAddToJobList = []
for item in devicesToUpgrade:
    device = session.get(externalApiURL + "devicesbyExternalId/" + item, cookies=filesUploaded.cookies, headers=headers, verify=False)
    if device.ok:
        #itemType: 0 for single device, itemType: 1 for device group
        devicesToAddToJobList += [{"itemId":(device.json()[0]['deviceId']), "itemType" : 0}]

### Next we will need to find the job we are looking for. It will match the name we put into jobToExecute at the begining of this script

In [11]:
jobs = session.get(externalApiURL + "/taskJobs", cookies=device.cookies, headers=headers, verify=False)

In [12]:
if jobs.ok:
    jobNumber = None
    for item in jobs.json():
        if item['name'] == jobToExecute:
            jobNumber = item['backup_JOB_ID']
    if jobNumber == None:
        print("Unable to find specified job")
        exit()

### Now that we found our job let's go grab that job

In [13]:
focusJob = session.get(externalApiURL + "/taskJob/" + str(jobNumber), cookies=jobs.cookies, headers=headers, verify=False)

### Our next step will be to replace the devices that were in the job with our devices. Then push the change back to BackBox

In [14]:
if focusJob.ok:
    editedJob = focusJob.json()
    editedJob["itemsIN_BackupJob"] = devicesToAddToJobList

In [15]:
updateJob = session.put(externalApiURL + "taskJob/", cookies=jobs.cookies,json=editedJob, verify=False)

In [16]:
if updateJob.ok:
    print("successfully added your devices to the update job")

successfully added your devices to the update job


### Our Last step will be to edit the Dynamic Fields of the job to point to the file we uploaded before. After this is done we can either run the API to run the job now, or we can run the job later at our convenience 

In [17]:
#get the data about what is currently in your dynamic fields
data = json.loads('[]')
headers = {"Accept": 'application/json'}
currentJobDynamicFields = session.put(internalApiURL + "tasks/jobs/getTaskJobDynamicFields/" + str(jobNumber), cookies=updateJob.cookies, headers=headers, json =data, verify=False)

In [18]:
#here we will edit the value of the file name that is in the IOS bin File UI field
editedDyanmicFields = currentJobDynamicFields.json()
for item in editedDyanmicFields:
    if item["uiName"] == "IOS bin File": # This is the UI name for the element in this specific job
        item['customValue'] = jobFileName

In [19]:
#Push edited data back to your dynamic fields
updateJobDynamicFields = session.post(internalApiURL + "tasks/jobs/updateJobDynamicFields/" + str(jobNumber), cookies=currentJobDynamicFields.cookies, headers=headers, json =editedDyanmicFields, verify=False)

In [20]:
if updateJobDynamicFields.ok:
    print("Success! You have updated the dynamic fields for " + jobToExecute + ".")

Success! You have updated the dynamic fields for UpgradeJob.


In [21]:
session.close()